In [1]:
import os


import torch
import torch.nn as nn
from torch.autograd import Variable

try:
    import torchinfo
    no_torchinfo = False
except ModuleNotFoundError:
    no_torchinfo = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch loaded; device is {device}")
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

torch loaded; device is cpu


In [2]:
class Net(nn.Module):

    """Defines the neural network model"""
    def __init__(self):

        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(4, 30).double()
        self.hidden_layer2 = nn.Linear(30, 30).double()
        self.hidden_layer3 = nn.Linear(30, 30).double()
        self.hidden_layer4 = nn.Linear(30, 10).double()
        self.hidden_layer5 = nn.Linear(10, 5).double()
        self.output_layer = nn.Linear(5, 1).double()

    """Defines the sequence of operations that are applied to the input tensors to produce the output of the neural network"""
    def forward(self, x, t, mean, variance):

        inputs = torch.cat([x,t,mean,variance], axis=1)
        layer1_out = torch.tanh(self.hidden_layer1(inputs))
        layer2_out = torch.tanh(self.hidden_layer2(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer3(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer4(layer3_out))
        layer5_out = torch.tanh(self.hidden_layer5(layer4_out))
        output = self.output_layer(layer5_out)
        return output

# def network_BC(t): #t to define u_theta
#     return t

In [3]:
class Discontinuity(torch.nn.Module):
    def __init__(self, a=1):

        super().__init__()
        self.c1 = torch.nn.Parameter(torch.randn(())) #pour que cela reste entre 0 et 1
        self.c2 = torch.nn.Parameter(torch.randn(()))
        self.c3 = torch.nn.Parameter(torch.randn(()))
        self.c4 = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.sparse_layer=False

    def x(self, t, x_0):

        return x_0 + self.c4 * t
    
    def forward(self, x, t ):

        x_t = self.x(t, x_0=0.5) #x_O c'est la position la discontinuité a gauche du creneau

        i1 = torch.where((0 <= x) & (x <= x_t), torch.tensor(1.0), torch.tensor(0.0))
        i2 = torch.where((x_t < x) & (x <= x_t + self.b), torch.tensor(1.0), torch.tensor(0.0))
        i3 = torch.where((x < x_t+self.b) & (x<= torch.tensor(1.0)), torch.tensor(1.0), torch.tensor(0.0))
        
        return  self.c1 * i1 + self.c2 * i2 + self.c3 * i3


In [4]:
class Trigo2(torch.nn.Module):
    def __init__(self,mu=0.0,sigma=1.0):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a1 = torch.nn.Parameter(torch.randn(())*sigma+mu)
        self.b1 = torch.nn.Parameter(torch.randn(())*sigma+mu)
        self.c1 = torch.nn.Parameter(torch.randn(())*sigma+mu)
        self.sparse_layer=False

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return torch.sin(self.a1*x) + self.c1*torch.cos(self.b1*x)

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = sin({self.a1.item()}x) + {self.c1.item()}cos({self.b1.item()} x)'